# Predicting review scores 

AI Black Belt - Yellow (June 2019).

---

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
df = pd.read_csv("data/amazon-reviews.csv")
df

## Text Feature Extraction with Bag-of-Words

In many tasks, like in the classical spam detection, your input data is text.
Free text with variables length is very far from the fixed length numeric representation that we need to do machine learning with scikit-learn.
However, there is an easy and effective way to go from text data to a numeric representation using the so-called bag-of-words model, which provides a data structure that is compatible with the machine learning aglorithms in scikit-learn.

<img src="figures/day3/bag_of_words.svg" width="100%">


Let's assume that each sample in your dataset is represented as one string, which could be just a sentence, an email, or a whole news article or book. To represent the sample, we first split the string into a list of tokens, which correspond to (somewhat normalized) words. A simple way to do this to just split by whitespace, and then lowercase the word. 

Then, we build a vocabulary of all tokens (lowercased words) that appear in our whole dataset. This is usually a very large vocabulary.
Finally, looking at our single sample, we could show how often each word in the vocabulary appears.
We represent our string by a vector, where each entry is how often a given word in the vocabulary appears in the string.

As each sample will only contain very few words, most entries will be zero, leading to a very high-dimensional but sparse representation.

The method is called "bag-of-words," as the order of the words is lost entirely.

In [ ]:
df = df[~df["Summary"].isnull()]

In [ ]:
X_raw = df["Summary"]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=500) # top 500 most frequent words
vectorizer.fit(X_raw)

In [ ]:
X_bag_of_words = vectorizer.transform(X_raw).toarray()

In [ ]:
wordCounts = zip(vectorizer.get_feature_names(), np.sum(X_bag_of_words, axis=0))
pd.DataFrame([{"word":word, "count":count} for word,count in wordCounts]).sort_values("count", ascending=False)

In [ ]:
vectorizer.inverse_transform(X_bag_of_words[:10])

## tf-idf Encoding


A useful transformation that is often applied to the bag-of-word encoding is the so-called term-frequency inverse-document-frequency (tf-idf) scaling, which is a non-linear transformation of the word counts.

The tf-idf encoding rescales words that are common to have less weight:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=500)
tfidf_vectorizer.fit(X_raw)

In [ ]:
X_tfidf = tfidf_vectorizer.transform(X_raw).toarray()

## Linear regression 


In [ ]:
from sklearn.model_selection import train_test_split

X = X_bag_of_words
y = df["Score"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
regressor = Ridge()
regressor.fit(X_train, y_train)

In [ ]:
wordCoef = pd.DataFrame({"word":list(vectorizer.get_feature_names()), "coef":regressor.coef_})
#wordCoef["abs"] = wordCoef["coef"].abs()
wordCoef = wordCoef.sort_values("coef", ascending=False)
wordCoef

In [ ]:
y_pred_train = regressor.predict(X_train)
regressor.score(X_test, y_test)

<div class="alert alert-success">
    <b>EXERCISE</b>:

Try to use the TF-IDF encoder to see if you obtain better results.
</div>

In [ ]:
# %load solutions/day3-04-01.py

<div class="alert alert-success">
    <b>EXERCISE</b>:

Predict the score for the test_sentences defined below.
</div>

In [ ]:
test_sentences = [
    "Great coffee. yummy",
    "This pork is disgusting",
    "dangerous and nasty",
    "look expired",
]

In [ ]:
# %load solutions/day3-04-02.py

<div class="alert alert-success">
    <b>EXERCISE</b>:

Try to use the review text instead of the review summary to see if we can improve the score.
</div>

<div class="alert alert-success">
    <b>EXERCISE</b>:

Try to combine the review text with the review summary to see if we can improve the score further.
</div>